# Описание проекта

Для защиты данных клиентов страховой компании разработаны методы преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Была проведена предобработка данных. Произведена проверка работы алгоритма модели линейной регрессии при перемножении на обратимую матрицу. Произведена проверка влияния перемножения на обратимую матрицу, а затем перемножения на обратную матрицу обратимой. Произведена проверка метрики `R2` Линейной регрессии на идентичных данных - сначала исходных, затем умноженных на обратимую матрицу, размер которой равен числу признаков. Метрики полностью совпали. Можно сделать вывод, что алгоритм работает.

### Задачи проекта
Методом преобразования данных защитить личную информацию клиентов страховой компании. При преобразовании данных качество моделей машинного обучения не должно ухудшиться. Подбирать наилучшую модель не требуется.

### Ключевые слова и навыки
`data science, machine learning, ML, Python, SQL, Git, Pandas, Numpy, Matplotlib, seaborn, Sklearn, Tableau, Spark, Hadoop, R, sci-py, Research, Kaggle, Algorithms, PyTorch, TensorFlow, CatBoost, xgboost, Support vector machines, градиентный бустинг, нелинейная оптимизация, кластеризация, random forest, descision trees,  regression,  Reinforcement Learning, OpenCV, PIL`

### Описание данных

Набор данных находится в файле `/datasets/insurance.csv`

Признаки: 
* `Пол`
* `Возраст`
* `Зарплата`
* `Члены семьи` – количество членов семьи

Целевой признак:
* `Страховые выплаты` – количество страховых выплат клиенту за последние 5 лет.
    
    
# Содержание <a name="title"></a>

[Импорт библиотек и задание констант](#import)
1. [Загрузка и подготовка данных](#1)
    
 
2. [Обоснование алгоритма преобразования](#2)

    
3. [Обучение моделей и проверка алгоритма ](#3)

    3.1 [Обучение модели на исходных данных](#3.1)
    
    3.2 [Преобразование признаков ](#3.2)
    
    3.3 [Обучение модели на преобразованных данных](#3.3)
    
    3.4 [Проверка качества алгоритма](#3.4)

# Импорт библиотек и задание констант <a name="import"></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

from IPython.display import display

RNG = 123 # используем константу в качестве random_state

In [2]:
class Explorer:
    class show(object):
        template = """<div style="float: left; padding: 10px;">
        <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
        </div>"""

        def __init__(self, *args):
            self.args = args
    
        def _repr_html_(self):
            return '\n'.join(self.template.format(a, eval(a)._repr_html_()) for a in self.args)
        
explorer = Explorer()

def review(data, i):
    return pd.concat([data.head(i), data.tail(i)], keys=['head', 'tail'])

[Содержание](#title)

## 1. Загрузка и подготовка данных <a name="1"></a>

In [3]:
df = pd.read_csv('/datasets/insurance.csv')

In [4]:
review(df, 4)

Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
head 0       1     41.0   49600.0            1                  0
     1       0     46.0   38000.0            1                  1
     2       0     29.0   21000.0            0                  0
     3       0     21.0   41700.0            2                  0
tail 4996    0     34.0   52400.0            1                  0
     4997    0     20.0   33900.0            2                  0
     4998    1     22.0   32700.0            3                  0
     4999    1     28.0   40600.0            1                  0

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Данные без пропусков. Типы данных соответствуют описанию

Исследуем на аномалии

In [6]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Аномалий не обнаружено

Проверим наличие дубликатов

In [7]:
print('Количество дубликатов', df.duplicated().sum())

Количество дубликатов 153


В датасете имеются дубликаты, проверим дубликаты на наличие закономерностей

In [8]:
review(df[df.duplicated()], 5)

Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
head 281     1     39.0   48100.0            1                  0
     488     1     24.0   32900.0            1                  0
     513     0     31.0   37400.0            2                  0
     718     1     22.0   32600.0            1                  0
     785     0     20.0   35800.0            0                  0
tail 4793    1     24.0   37800.0            0                  0
     4902    1     35.0   38700.0            1                  0
     4935    1     19.0   32700.0            0                  0
     4945    1     21.0   45800.0            0                  0
     4965    0     22.0   40100.0            1                  0

Очевидные закономерности не прослеживаются. Совпадения признаков возможны, но для надёжности удалим дубликаты.

In [9]:
df = df.drop_duplicates().reset_index(drop=True)

In [10]:
review(df, 4)

Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
head 0       1     41.0   49600.0            1                  0
     1       0     46.0   38000.0            1                  1
     2       0     29.0   21000.0            0                  0
     3       0     21.0   41700.0            2                  0
tail 4843    0     34.0   52400.0            1                  0
     4844    0     20.0   33900.0            2                  0
     4845    1     22.0   32700.0            3                  0
     4846    1     28.0   40600.0            1                  0

[Содержание](#title)

## 2. Обоснование алгоритма преобразования <a name="2"></a>

Для защиты данных предполагается домножить матрицу признаков на обратимую матрицу. Ответим на вопрос **"Изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу?"**.

**Вводные данные**

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $A$ – обратимая матрица

- $A^{-1}$ – обратная матрица

- $E$ – единичная матрица

- $MSE$ – Mean squared error

- $B, D$ – квадратные матрицы

$$
A A^{-1} = A^{-1} A = E
$$

$$
(A^{-1})^{T} A^{T} = E
$$

$$
X E = E X = X
$$

$$ 
BD^{-1} = D^{-1} B^{-1}
$$

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Докажем, что при домножении признаков на обратимую матрицу данные можно восстановить

$$
X E = X A A^{-1} = X
$$

Докажем, что $a = a^{`}$  и, следовательно, качество предсказания не изменится

$$
a^{`} = XA((XA)^T XA)^{-1} (XA)^T y = XA(A^T X^T XA)^{-1} A^T X^T y =
$$

$$
XA(X^T XA)^{-1} (A^T)^{-1} A^T X^T y = XAA^{-1}(X^{T}X)^{-1} E X^{T} y = 
$$

$$
= XE(X^{T}X)^{-1}X^{T} y = X(X^{T}X)^{-1}X^{T} y = a
$$

Что и требовалось доказать.

Таким образом, **качество линейной регрессии при умножении признаков на обратимую матрицу не изменится**. Для защиты признаков домножим матрицу признаков на обратимую матрицу.

[Содержание](#title)

## 3. Обучение моделей и проверка алгоритма <a name="3"></a>

Обучим две модели логистической регрессии:
* модель на исходных данных;
* модель на зашифрованных данных.

Произведем разделение данных на обучающую и тестовую выборку.

In [11]:
target = df['Страховые выплаты']
features = df.drop('Страховые выплаты', axis=1)

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=RNG)

[Содержание](#title)

### 3.1 Обучение модели на исходных данных <a name="3.1"></a>

In [12]:
model_logistic = LinearRegression()
model_logistic.fit(features_train, target_train)
predicted_true = model_logistic.predict(features_test)
r2_true = r2_score(target_test, predicted_true)
print('R2 на исходных данных равен', r2_true)

R2 на исходных данных равен 0.40978958053663006


[Содержание](#title)

### 3.2 Преобразование признаков <a name="3.2"></a>

Для преобразования признаков домножим их на обратимую матрицу 4 на 4. Сформируем матрицу

In [13]:
i = 0
while i < 1:
    try:
        inv_matrix = np.random.normal(5, 10, size=(4, 4))
        np.linalg.inv(inv_matrix)
        print('Матрица сформирована')
        i += 1
    except:
        i = 0

Матрица сформирована


In [14]:
features_train_inv = pd.DataFrame(np.dot(np.array(features_train), inv_matrix))
features_test_inv = pd.DataFrame(np.dot(np.array(features_test), inv_matrix))

Проверим новую матрицу признаков

In [15]:
review(features_train_inv, 4)

0              1             2              3
head 0    -29933.374560  187629.005162  6.634866e+05  116671.944212
     1    -22521.957387  141040.091629  4.983140e+05   87699.472059
     2    -33502.729681  210140.545380  7.429264e+05  130681.142876
     3    -28782.249727  180077.139911  6.359663e+05  111983.811583
tail 3631 -31206.130732  195585.508073  6.914482e+05  121618.425477
     3632 -32806.702616  205389.257896  7.259633e+05  127714.073238
     3633 -46691.061850  292632.976962  1.034944e+06  181966.762271
     3634 -28077.843420  175690.038572  6.208098e+05  109245.704238

Проверим возможно ли их восстановить

In [16]:
test_inv = pd.DataFrame(np.dot(np.array(features_train_inv), np.linalg.inv(inv_matrix)))

# округлим и возьмём модуль
test_inv = np.abs(np.round_(test_inv, decimals=0, out=None) )

display(explorer.show('review(features_train, 4)', 'review(test_inv, 4)'))

Данные восстановлены.

[Содержание](#title)

### 3.3 Обучение модели на преобразованных данных <a name="3.3"></a>

In [17]:
model_logistic = LinearRegression()
model_logistic.fit(features_train_inv, target_train)
predicted_inv = model_logistic.predict(features_test_inv)
r2_inv = r2_score(target_test, predicted_inv)
print('R2 на преобразованных данных равен', r2_inv)

R2 на преобразованных данных равен 0.40978958053656933


[Содержание](#title)

### 3.4 Проверка качества алгоритма <a name="3.4"></a>

In [18]:
print('R2 на исходных данных равен', r2_true)
print('R2 на преобразованных данных равен', r2_inv)

R2 на исходных данных равен 0.40978958053663006
R2 на преобразованных данных равен 0.40978958053656933


**Вывод**

Метрика **R2** практически не изменилась, малейшие погрешности дают алгоритмы округления Python. **Метод** защиты данных путём умножения их на обратимую матрицу **работоспособен**.

К преимуществам данного подхода можно отнести, в первую очередь, ощутимую экономию ресурсов по сравнению с некоторыми известными алгоритмами шифрования, в особенности это касается ассиметричных алгоритмов с достаточно длинными ключами (AES, RSA), которые требуют много процессорного времени.

К главному недостатку подхода можно отнести увеличение сложности проверки на вырожденность преобразования с ростом количества признаков в выборке. В рассмотренном случае необходимо подобрать подходящую матрицу размера  4×4 . Однако, если признаки будут исчисляться тысячами, что в теории всегда надо положить, то расчет соответствующих определителей может быть крайне ресурсозатратным.

[Содержание](#title)